In [1]:
import os
import pandas as pd
import config
from utils.dataset import chest_xray_datasplit, ChestXrayDataset, ChestXrayDatasetWithMask
from utils.model import EfficientNetV2_MultiLabel
from utils.train import train, validate, compute_pos_weight

In [2]:
test_path = os.path.join(config.DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_test.csv')
train_path = os.path.join(config.DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_train.csv')
val_path = os.path.join(config.DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_val.csv')

# Load all CSVs
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

# Combine them
full_df = pd.concat([df_train, df_val, df_test], ignore_index=True)

In [3]:
['Pneumothorax', 'Emphysema', 'Subcutaneous Emphysema', 'Pneumoperitoneum', 'Pneumomediastinum'].extend(['id', 'No Finding', 'subj_id'])

In [4]:
columns = ['Pneumothorax', 'Emphysema', 'Subcutaneous Emphysema', 'Pneumoperitoneum', 'Pneumomediastinum']
classes_df = full_df[columns]
classes_df = classes_df[classes_df.sum(axis=1) > 0]
columns.extend(['id', 'No Finding', 'subj_id'])
# Filter full_df based on the index of the filtered classes_df
group1_df = full_df.loc[classes_df.index, columns]
#group1_df = full_df[columns]

In [5]:
from torch.utils.data import DataLoader
full_dataset = ChestXrayDataset(dataframe=group1_df, img_dir=os.path.join(config.DATASET_DIR, 'cxr', 'images'))

train_dataset, val_dataset, test_dataset = chest_xray_datasplit(group1_df, full_dataset, dataset_dir=os.path.join(config.DATASET_DIR, 'cxr', 'images'))

train_loader = DataLoader(train_dataset, batch_size=96, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=96, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=96, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
# Initialize model
model = EfficientNetV2_MultiLabel(num_classes=len(classes_df.columns))

In [7]:
import torch
def compute_pos_weight_from_df(full_df, label_cols):
    num_samples = len(full_df)

    # Sum positives per class
    pos_counts = full_df[label_cols].sum().values  # shape [num_classes]
    neg_counts = num_samples - pos_counts

    pos_weight = neg_counts / (pos_counts + 1e-6)  # avoid div by zero
    return torch.tensor(pos_weight, dtype=torch.float32)

pos_weight = compute_pos_weight_from_df(classes_df, list(classes_df.columns))

In [8]:
import torch
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

# Train
train(model, train_loader, val_loader, device, epochs=10, lr=1e-4, save_path=config.MODEL_FOLDER, file_name="PulmoScanX_Group1_Effnet_v2", pos_weight=pos_weight,
      multi_gpu=True, device_ids=[1,2,3])

Using 3 GPUs: [1, 2, 3]


Epoch 1/10: 100%|██████████| 56/56 [01:57<00:00,  2.10s/it, loss=0.642]


Epoch 1 Training Loss: 0.9523


Validation Loss: 0.9146
F1: 0.3726 | AUC: 0.6314 | Accuracy: 0.1539



Epoch 2/10: 100%|██████████| 56/56 [01:50<00:00,  1.97s/it, loss=0.665]

...


Epoch 9 Training Loss: 0.4076


Validation Loss: 0.6317
F1: 0.5911 | AUC: 0.8005 | Accuracy: 0.4837



Epoch 10/10: 100%|██████████| 56/56 [01:49<00:00,  1.96s/it, loss=0.902]


Epoch 10 Training Loss: 0.3891


Validation Loss: 0.7426
F1: 0.6822 | AUC: 0.8065 | Accuracy: 0.6174



In [12]:
from utils.evaluate import evaluate, evaluate_per_class, evaluate_per_class2
model = EfficientNetV2_MultiLabel(num_classes=len(classes_df.columns))
model.load_state_dict(torch.load('../models/PulmoScanX_Group1_Effnet_v2_checkpoint.pth'))
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
model.to(device)
evals_per_class = evaluate_per_class(model, test_loader, device, list(classes_df.columns))


Per-Class F1 Scores (with tuned thresholds):
             Pathology  F1 Score  Threshold
          Pneumothorax  0.842424   0.261163
Subcutaneous Emphysema  0.712460   0.707289
             Emphysema  0.683168   0.588021
      Pneumoperitoneum  0.592593   0.799302
     Pneumomediastinum  0.488889   0.953161
